tor.exe -f torrc

In [1]:
%reset -f
import requests

# Set up Tor proxy
tor_proxy = "socks5://127.0.0.1:9050"
proxies = {
    "http": tor_proxy,
    "https": tor_proxy,
}

# Check if Tor is working by making a request to the official Tor check website
try:
    response = requests.get("https://check.torproject.org", proxies=proxies)
    if "Congratulations. This browser is configured to use Tor." in response.text:
        print("Tor is working.")
    else:
        print("Tor is not working.")
except requests.exceptions.RequestException as e:
    print("Tor is not working:", e)

Tor is working.


In [2]:
#%pip install stem

Test commente Scrap 

In [2]:
# Ce que fait le code Python (canevas explicatif)

# 1. Suppression de toutes les variables en mémoire (utile dans un notebook)
%reset -f

# 2. Importation des bibliothèques nécessaires
from datetime import datetime
import os
import hashlib
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

# 3. Configuration du proxy Tor
# Tor doit être en cours d'exécution sur le port 9050
# Le trafic Web passera par le réseau Tor pour garantir l'anonymat

# Création d'une URL de proxy SOCKS5 pointant vers le client Tor
tor_proxy = "socks5://127.0.0.1:9050"

# 4. Configuration de Chrome pour utiliser le proxy Tor
chrome_options = Options()
chrome_options.add_argument(f"--proxy-server={tor_proxy}")

# 5. Lancement de Chrome avec les options spécifiées
# Utilise WebDriver Manager pour installer automatiquement le bon driver

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 6. Navigation vers une publication Facebook précise
url_post = "https://www.facebook.com/Hespress/posts/pfbid021ekyH7772jfUY2QgJm4DUSxjf3EqPrjvHL3851jfASVYuUHRubxmWtustmc8S1Asl"
driver.get(url_post)

# 7. Pause manuelle pour permettre à l'utilisateur d'accepter les cookies ou de se connecter si besoin
input("Press Enter to continue...")


# Click the <div> element with 'Allow all cookies'
# allow_cookies_span = driver.find_element(By.XPATH, '//span[contains(text(), "Allow all cookies")]')
# ActionChains(driver).move_to_element(allow_cookies_span).click().perform()

# 8. Cliquer sur tous les boutons "See more" pour déployer le texte complet des commentaires
see_more_divs = driver.find_elements(By.XPATH, '//div[text()="See more"]')
for div in see_more_divs:
    div.click()

# 9. Attendre 5 secondes pour laisser le temps au DOM de se mettre à jour
# (devrait être remplacé par WebDriverWait pour plus de robustesse)
time.sleep(5)

# 10. Extraction de tous les <div> contenant un style avec "text-align:" (correspond aux commentaires)
div_elements = driver.find_elements(By.XPATH, '//div[contains(@style, "text-align:")]')

# 11. Récupération du texte brut de tous ces divs
all_text = [div.text for div in div_elements]

# 12. Sauvegarde des commentaires extraits dans un fichier texte
# Le nom de fichier est basé sur un hash MD5 du contenu + la date
current_date = datetime.now().strftime("%Y-%m-%d")
output_dir = "./scraped_data"
os.makedirs(output_dir, exist_ok=True)

content_hash = hashlib.md5("\n".join(all_text).encode()).hexdigest()
output_file = os.path.join(output_dir, f"{content_hash}_{current_date}.txt")

with open(output_file, "w", encoding="utf-8") as file:
    file.write("\n".join(all_text))

print(f"Data saved to {output_file}")

# 13. Fermeture du navigateur
driver.quit()


Data saved to ./scraped_data\4b506db29e284d3ede8082abc1ba9baa_2025-05-21.txt


In [4]:

from stem.control import Controller
with Controller.from_port(port=9051) as c:
    c.authenticate()          # Cookie auth
    print("Tor version:", c.get_version())   # devrait afficher la version
    print("Tor bytes read:", c.get_info("traffic/read"))
    print("Tor bytes written:", c.get_info("traffic/written"))

Tor version: 0.4.8.16 (git-64ccafd8115ecdec)
Tor bytes read: 73165353
Tor bytes written: 4618400


In [5]:
# %pip install undetected-chromedriver
# %pip install facebook-scraper
# %pip install requests 
# %pip install lxml_html_clean
# %pip install lxml
# %pip install pandas

In [6]:
%pip install lxml_html_clean

Note: you may need to restart the kernel to use updated packages.


In [7]:
# Scraper URLs Facebook – **facebook‑scraper** + Tor + Cookies  
# -------------------------------------------------------------
# Objectif : éviter le blocage « You’re Temporarily Blocked » par Facebook.  
# Méthode : on abandonne Selenium ; on utilise la librairie `facebook_scraper`,  
# qui fait de simples requêtes HTTP (moins détectable).  
# On passe toujours par le proxy Tor, ET on fournit ses propres cookies Facebook  
# (ainsi le site considère la session comme authentifiée et stable).

"""
Étapes préalables :
1. Dans Chrome (ou Firefox), connecté à Facebook, exportez les cookies :
   – installez l’extension **EditThisCookie** ou **Get cookies.txt**.  
   – exportez et enregistrez dans `cookies_fb.txt` (fichier Netscape).  
2. Placez ce fichier dans le même répertoire que ce script.
3. (Facultatif) Limitez le nombre de pages (`page_limit`) pour rester discret.
"""

import csv, datetime, os, time
from facebook_scraper import get_posts
import requests

# ---------- paramètres ----------
PAGE_NAME   = "tawjihhoussam"        # identifiant de la page
PAGE_LIMIT  = 5                       # nombre de pages « scroll » (≈ 50 posts/page)
COOKIE_FILE = "cookies_fb.txt"       # exporté depuis le navigateur
CSV_DIR     = "scraped_data"
PROXY_TOR   = "socks5://127.0.0.1:9050"

# ---------- proxies pour facebook_scraper ----------
proxies = {
    "http":  PROXY_TOR,
    "https": PROXY_TOR,
}

print("[+] Test IP Tor actuelle…")
ip = requests.get("https://api.ipify.org", proxies=proxies, timeout=10).text
print("    → IP :", ip)

print(f"[+] Collecte des posts de {PAGE_NAME} via facebook_scraper…")
links = set()
for post in get_posts(PAGE_NAME, cookies=COOKIE_FILE, extra_info=False,
                      page_limit=PAGE_LIMIT, proxies=proxies):
    links.add(post["post_url"].split("?", 1)[0])
    time.sleep(2)   # 1 requête toutes les 2 s ≈ 30/h (reste sous le rate‑limit)

os.makedirs(CSV_DIR, exist_ok=True)
csv_path = os.path.join(CSV_DIR, f"post_links_{datetime.date.today()}.csv")
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerows([["url"]] + [[u] for u in sorted(links)])

print(f"✅ {len(links)} URLs enregistrées → {csv_path}")
print("[+] Fin du script.")
 

[+] Test IP Tor actuelle…
    → IP : 195.47.238.88
[+] Collecte des posts de tawjihhoussam via facebook_scraper…
✅ 0 URLs enregistrées → scraped_data\post_links_2025-05-19.csv
[+] Fin du script.


In [8]:
# Scraper URLs Facebook – **facebook‑scraper** + Tor + Cookies  
# -------------------------------------------------------------
# Objectif : éviter le blocage « You’re Temporarily Blocked » par Facebook.  
# Méthode : on abandonne Selenium ; on utilise la librairie `facebook_scraper`,  
# qui fait de simples requêtes HTTP (moins détectable).  
# On passe toujours par le proxy Tor, ET on fournit ses propres cookies Facebook  
# (ainsi le site considère la session comme authentifiée et stable).
#
# ⚠️ Dépendances minimales :
#   pip install "facebook_scraper[html]"    # installe aussi lxml_html_clean
#   OU bien :  pip install facebook_scraper lxml_html_clean requests
#
# ────────────────────────────────────────────────────────────────────────
"""
Étapes préalables :
1. Dans Chrome (ou Firefox), connecté à Facebook, exportez les cookies :
   – installez l’extension **EditThisCookie** ou **Get cookies.txt**.  
   – exportez et enregistrez dans `cookies_fb.txt` (fichier Netscape).  
2. Placez ce fichier dans le même répertoire que ce script.
3. (Facultatif) Limitez le nombre de pages (`PAGE_LIMIT`) pour rester discret.
"""

import csv, datetime, os, time, subprocess, sys

# ----- installer lxml_html_clean à la volée si absent -----
try:
    from lxml.html.clean import Cleaner  # noqa: F401 (test import)
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "lxml_html_clean"])

from facebook_scraper import get_posts
import requests

# ---------- paramètres ----------
PAGE_NAME   = "tawjihhoussam"        # identifiant de la page
PAGE_LIMIT  = 5                       # nombre de pages « scroll » (≈ 50 posts/page)
COOKIE_FILE = "cookies_fb.txt"       # exporté depuis le navigateur
CSV_DIR     = "scraped_data"
PROXY_TOR   = "socks5://127.0.0.1:9050"

proxies = {
    "http":  PROXY_TOR,
    "https": PROXY_TOR,
}

print("[+] Test IP Tor actuelle…")
try:
    ip = requests.get("https://api.ipify.org", proxies=proxies, timeout=10).text
    print("    → IP :", ip)
except Exception as e:
    print("    Impossible de récupérer l'IP via Tor :", e)

print(f"[+] Collecte des posts de {PAGE_NAME} via facebook_scraper…")
links = set()
for post in get_posts(PAGE_NAME, cookies=COOKIE_FILE, extra_info=False,
                      page_limit=PAGE_LIMIT, proxies=proxies):
    links.add(post["post_url"].split("?", 1)[0])
    time.sleep(2)   # ≈30 requêtes/h, on reste loin du blocage

os.makedirs(CSV_DIR, exist_ok=True)
csv_path = os.path.join(CSV_DIR, f"post_links_{datetime.date.today()}.csv")
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerows([["url"]] + [[u] for u in sorted(links)])

print(f"✅ {len(links)} URLs enregistrées → {csv_path}")


[+] Test IP Tor actuelle…
    → IP : 195.47.238.88
[+] Collecte des posts de tawjihhoussam via facebook_scraper…
✅ 0 URLs enregistrées → scraped_data\post_links_2025-05-19.csv
